In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
import numpy as np 
from my_fft import FFTPower, Mesh 

### Load Data 

In [2]:
data = np.load("test_data.pkl.npy")
print(data.shape)
print(data.dtype)

(100000,)
[('Position', '<f8', (3,)), ('Weight', '<f8')]


### Do cic and FFT with compensating filter

In [3]:
mesh = Mesh(Nmesh=100, BoxSize=1000.0)
cic_mesh = mesh.run_cic(data, weight="Weight", norm=False, nthreads=3) # For PS of overdensity, you can set norm=True
mesh_complex = mesh.r2c(cic_mesh, compensated=True, nthreads=2)

In [4]:
mesh.attrs

{'Nmesh': array([100, 100, 100], dtype=int32),
 'BoxSize': array([1000., 1000., 1000.]),
 'N': 100000,
 'W': 49926.69584217917,
 'W2': 33220.87005622538,
 'shotnoise': 13327.39752263791,
 'num_per_cell': 0.049926695842179174,
 'compensated': True}

### Calculate FFTPower with 1d(Pk) and 2d(Pkmu)

In [5]:
fftpower = FFTPower(Nmesh=100, BoxSize=1000.0, shotnoise=mesh.attrs["shotnoise"]) # Nmesh and BoxSize you need keep the same as Mesh you used

In [8]:
power = fftpower.run(mesh_complex, kmin=0.1, kmax=0.5, dk=0.02, mode="1d", nthreads=2, Nmu=5, run_ps_3d=True) # mode can be 1d and 2d. run_ps_3d should be set True at once. If you run fftpower on the same complex field, such as call fftpower.runPS3D before, you must set run_ps_3d=False

True True


In [7]:
fftpower.is_run_ps_3d

True

In [11]:
# You can save and load fftpower with methods: save and load. Note that "load" is a class method.
# fftpower.save("test_power_1D.pkl")
fftpower_check = FFTPower.load("test_power_1D_check.pkl")

In [9]:
power["Pk"]

array([[33.23062651+0.j],
       [33.09789983+0.j],
       [33.23244369+0.j],
       [32.89105631+0.j],
       [33.24854718+0.j],
       [33.24793627+0.j],
       [33.50166451+0.j],
       [33.10905697+0.j],
       [32.89698503+0.j],
       [33.10298633+0.j],
       [33.28094685+0.j],
       [33.15231079+0.j],
       [33.39592625+0.j],
       [33.26706628+0.j],
       [33.1400421 +0.j],
       [32.98775348+0.j],
       [33.51274413+0.j],
       [33.23725829+0.j],
       [33.79554019+0.j]])

In [12]:
fftpower_check.power["Pk"].dtype

dtype('complex128')